In [2]:
import os
from datetime import datetime

import googlemaps as gmaps
from vvspy import get_departures, get_trips
from vvspy.obj import Departure, Trip

In [3]:
station_1 = "de:08111:355"  # Wallgraben
station_2 = "de:08111:6002"  # Vaihingen
station_3 = "de:08111:6112"  # Hauptbahnhof
station_4 = "de:08111:6056"  # Stadtmitte

In [4]:
"""
Get specific data about incoming departure 
and e.g. display them on a monitor
"""

result = get_departures(station_2, limit=10)

if result:
    for res in result:
        print(res)

[18:22] [84]: Vaihingen ZOB - Sindelfingen
[18:22] [86]: Vaihingen ZOB - Leinfelden Bf
[18:24] [81]: Lambertweg - Büsnau
[18:29] [U1]: Vaihingen Bf - Fellbach
[Delayed] [18:31] [S3]: Backnang - Flughafen/Messe
[Delayed] [18:31] [S3]: Flughafen/Messe - Backnang
[18:31] [U3]: Vaihingen Bf - Plieningen
[18:33] [81]: Büsnauer Platz - Dürrlewang
[18:36] [82]: Waldeck - Rohr Mitte
[18:36] [82]: Rohr Mitte - Waldeck


In [5]:
result = get_trips(station_4, station_1, limit=10)

if result:
    for res in result:
        print(res)

Connection (14 minutes):
[S2]: [Delayed] [17:29] @ Stadtmitte - [Delayed] [17:41] @ Vaihingen
[U3]: [17:46] @ Vaihingen Bf - [17:48] @ Wallgraben
Connection (13 minutes):
[S1]: [Delayed] [17:38] @ Stadtmitte - [17:49] @ Vaihingen
[U3]: [18:01] @ Vaihingen Bf - [18:03] @ Wallgraben
Connection (21 minutes):
[43]: [17:43] @ Rotebühlplatz - [17:47] @ Charlottenplatz
[U12]: [17:50] @ Charlottenplatz - [18:07] @ Wallgraben
Connection (14 minutes):
[S2]: [Delayed] [17:59] @ Stadtmitte - [Delayed] [18:11] @ Vaihingen
[U3]: [18:16] @ Vaihingen Bf - [18:18] @ Wallgraben


In [6]:
from datetime import timedelta
from typing import List

from requests import Response


result = get_trips(station_4, station_1, check_time=datetime.now()+timedelta(hours=2), limit=10)

if result is not None and not isinstance(result, Response):
    for res in result:
        print(f"Duration: {int(res.duration/ 60)} (not accurate)")
        for connection in res.connections:
            print(f"With {connection.transportation.disassembled_name} from {connection.origin.name} at {connection.origin.departure_time_estimated} to {connection.destination.name} at {connection.destination.arrival_time_estimated}")
        print(f"Real Duration: {int((res.connections[-1].destination.arrival_time_estimated - res.connections[0].origin.departure_time_estimated).total_seconds()/60)}")

Duration: 14 (not accurate)
With S2 from Stadtmitte at 2022-12-04 19:27:00 to Vaihingen at 2022-12-04 19:39:00
With U3 from Vaihingen Bf at 2022-12-04 19:46:00 to Wallgraben at 2022-12-04 19:48:24
Real Duration: 21
Duration: 21 (not accurate)
With 43 from Rotebühlplatz at 2022-12-04 19:28:00 to Charlottenplatz at 2022-12-04 19:32:00
With U12 from Charlottenplatz at 2022-12-04 19:35:00 to Wallgraben at 2022-12-04 19:52:00
Real Duration: 24
Duration: 14 (not accurate)
With S1 from Stadtmitte at 2022-12-04 19:37:00 to Vaihingen at 2022-12-04 19:49:00
With U3 from Vaihingen Bf at 2022-12-04 20:01:00 to Wallgraben at 2022-12-04 20:03:24
Real Duration: 26
Duration: 21 (not accurate)
With 43 from Rotebühlplatz at 2022-12-04 19:43:00 to Charlottenplatz at 2022-12-04 19:47:00
With U12 from Charlottenplatz at 2022-12-04 19:50:00 to Wallgraben at 2022-12-04 20:07:00
Real Duration: 24


In [7]:
from aswe.api.navigation.vvs import get_next_connection, get_latest_connection

trip_1 = get_next_connection(station_4, station_1)
trip_2 = get_latest_connection(station_4, station_1, datetime.now()+timedelta(hours=3))

print(f"Trip 1:")
print(f"Duration: {trip_1.duration}")
for con in trip_1.connections:
    print(f"Connection: {con.train_name} {con.start_location} to {con.end_location} [{con.start_time.strftime('%H:%M')} - {con.end_time.strftime('%H:%M')}]")

print(f"Trip 2:")
print(f"Duration: {trip_2.duration}")
for con in trip_2.connections:
    print(f"Connection: {con.train_name} {con.start_location} to {con.end_location} [{con.start_time.strftime('%H:%M')} - {con.end_time.strftime('%H:%M')}]")


Trip 1:
Duration: 19
Connection: S2 Stadtmitte to Vaihingen [18:29 - 18:41]
Connection: U3 Vaihingen Bf to Wallgraben [18:46 - 18:48]
Trip 2:
Duration: 24
Connection: 43 Rotebühlplatz to Charlottenplatz [20:43 - 20:47]
Connection: U12 Charlottenplatz to Wallgraben [20:50 - 21:07]


In [8]:
client = gmaps.Client(key=os.getenv("GOOGLE_MAPS_API_KEY"))

In [19]:
# modes: "driving", "walking", "bicycling" or "transit"

now = datetime.now()
directions_result = client.directions("Sydney Town Hall", "Parramatta, NSW", arrival_time=datetime.now(), mode="bicycling")

print(f"Distance: {directions_result[0]['legs'][0]['distance']['value']} meter")
print(f"Time needed: {int(directions_result[0]['legs'][0]['duration']['value']/60)} minutes")

directions_result

Distance: 28738 meter
Time needed: 99 minutes


[{'bounds': {'northeast': {'lat': -33.811346, 'lng': 151.2061774},
   'southwest': {'lat': -33.8727676, 'lng': 151.0017323}},
  'copyrights': 'Map data ©2022 Google',
  'legs': [{'distance': {'text': '28.7 km', 'value': 28738},
    'duration': {'text': '1 hour 39 mins', 'value': 5960},
    'end_address': 'Parramatta NSW 2150, Australia',
    'end_location': {'lat': -33.8147745, 'lng': 151.0017323},
    'start_address': '483 George St, Sydney NSW 2000, Australia',
    'start_location': {'lat': -33.8727676, 'lng': 151.2061774},
    'steps': [{'distance': {'text': '44 m', 'value': 44},
      'duration': {'text': '1 min', 'value': 6},
      'end_location': {'lat': -33.8727466, 'lng': 151.2057033},
      'html_instructions': 'Head <b>west</b> on <b>Druitt St</b> toward <b>Market Row</b>',
      'polyline': {'points': 'xwvmEsm{y[CZAb@@^'},
      'start_location': {'lat': -33.8727676, 'lng': 151.2061774},
      'travel_mode': 'BICYCLING'},
     {'distance': {'text': '0.4 km', 'value': 449},
 